# Algorytmy ewolucyjne i metaheurystyczne - Heurystyki konstrukcyjne

**Install libs**

In [55]:
!pip install requests
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip install scipy
!pip install sklearn
!pip install pandoc

  Using cached matplotlib-3.3.4-cp38-cp38-win_amd64.whl (8.5 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp38-cp38-win_amd64.whl (51 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


**Get data data:**

In [51]:
import requests

target_url_kroa100 = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/kroA100.tsp"
target_url_krob100 = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/kroB100.tsp"

def get_file_from_url(target_url):
    file = requests.get(target_url)
    return file.text
    

**Output test:**

In [67]:
data = get_file_from_url(target_url_kroa100)
for line in data.split("\n")[6:-2]:
    print(line.split())

['1', '1380', '939']
['2', '2848', '96']
['3', '3510', '1671']
['4', '457', '334']
['5', '3888', '666']
['6', '984', '965']
['7', '2721', '1482']
['8', '1286', '525']
['9', '2716', '1432']
['10', '738', '1325']
['11', '1251', '1832']
['12', '2728', '1698']
['13', '3815', '169']
['14', '3683', '1533']
['15', '1247', '1945']
['16', '123', '862']
['17', '1234', '1946']
['18', '252', '1240']
['19', '611', '673']
['20', '2576', '1676']
['21', '928', '1700']
['22', '53', '857']
['23', '1807', '1711']
['24', '274', '1420']
['25', '2574', '946']
['26', '178', '24']
['27', '2678', '1825']
['28', '1795', '962']
['29', '3384', '1498']
['30', '3520', '1079']
['31', '1256', '61']
['32', '1424', '1728']
['33', '3913', '192']
['34', '3085', '1528']
['35', '2573', '1969']
['36', '463', '1670']
['37', '3875', '598']
['38', '298', '1513']
['39', '3479', '821']
['40', '2542', '236']
['41', '3955', '1743']
['42', '1323', '280']
['43', '3447', '1830']
['44', '2936', '337']
['45', '1621', '1830']
['46', '33

In [68]:
def get_lat_lon(list_of_cords):
    return zip(*[(int(i.split()[1]), int(i.split()[2])) for i in list_of_cords])

cords = data.split("\n")[6:-2]
lat, lon = get_lat_lon(cords)
assert len(lat) == 100
assert len(lon) == 100

**Calculate matrix**

In [69]:
import pandas as pd
import numpy as np
import math

def calc_matrix(lat, lon):
    matrix_size = len(lat)
    matrix = pd.DataFrame(0, index=np.arange(matrix_size), columns=np.arange(matrix_size))
    for row in range(matrix.shape[0]):
        for col in range(row, matrix.shape[1]):
            if row == col:
                matrix.iat[row, col] = 0
            else:
                matrix.iat[row, col] = matrix.iat[col, row] =  round(math.sqrt(((lat[col]-lat[row])**2)+((lon[col]-lon[row])**2)))
    
    return matrix
test = calc_matrix(lat,lon)
print(test)

      0     1     2     3     4     5     6     7     8     9   ...    90  \
0      0  1693  2252  1104  2523   397  1447   425  1424   749  ...   753   
1   1693     0  1708  2403  1186  2057  1392  1620  1343  2442  ...  1978   
2   2252  1708     0  3333  1074  2623   811  2502   829  2794  ...  1885   
3   1104  2403  3333     0  3447   822  2538   851  2512  1030  ...  1760   
4   2523  1186  1074  3447     0  2919  1424  2606  1400  3218  ...  2468   
..   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
95  2377  1201   826  3344   253  2773  1196  2496  1176  3050  ...  2266   
96   815  2093  3018   324  3124   610  2230   527  2201   934  ...  1519   
97   783  1911  1786  1821  2374  1003  1010  1200  1014  1036  ...    99   
98  1468  3156  3315  1499  3863  1154  2544  1684  2546   726  ...  1436   
99  2643  1831   454  3701   894  3025  1231  2857  1240  3220  ...  2327   

      91    92    93    94    95    96    97    98    99  
0    303   413  